In [1]:
import pandas as pd

In [28]:
df = pd.read_csv('data/stock.csv', header=None)
df.rename(columns={0: 'Stock Name', 
					1: 'Previous Close', 
					2: 'Current Price', 
					3: 'Minimum(Day)', 
					4: 'Maximum(Day)',
					5: 'Minimum Threshold',
					6: 'Maximum Threshold',
					7: 'DateNTime',
					8: 'difference'}, inplace=True)

df['DateNTime'] = pd.to_datetime(df['DateNTime'])
df = df.round(2)
df['Up/Down'] = df['difference'].map(lambda x: 'Up' if x > 0 else 'Down')

In [29]:
sample = df.groupby(['Stock Name'])['DateNTime'].max().reset_index()

,Stock Name,DateNTime
0,Birlasoft Ltd,2022-02-09 10:30:38
1,Coforge Ltd,2022-02-09 10:30:44
2,Infosys Ltd,2022-02-09 10:30:41
3,Jubilant Ingrevia Ltd,2022-02-09 10:30:40
4,KPIT Technologies Ltd,2022-02-09 10:30:39
5,Steel Authority of India Limited,2022-02-09 10:30:37
6,Tata Motors Limited Fully Paid Ord. Shrs,2022-02-09 10:30:43
7,Tata Power Company Limited,2022-02-09 10:30:42
8,Trident Ltd,2022-02-09 10:30:38
9,Vodafone Idea Ltd,2022-02-09 10:30:36


In [30]:
df[df['DateNTime'].isin(sample['DateNTime'])]

,Stock Name,Previous Close,Current Price,Minimum(Day),Maximum(Day),Minimum Threshold,Maximum Threshold,DateNTime,difference,Up/Down
273,Vodafone Idea Ltd,10.85,10.95,10.90,11.15,10.5,11.5,2022-02-09 10:30:36,0.10,Up
274,Steel Authority of India Limited,101.90,101.55,100.55,102.80,100.0,130.0,2022-02-09 10:30:37,-0.35,Down
275,Birlasoft Ltd,442.55,455.55,445.55,464.00,440.0,500.0,2022-02-09 10:30:38,13.00,Up
276,Trident Ltd,60.45,59.70,59.50,61.50,59.5,70.0,2022-02-09 10:30:38,-0.75,Down
277,KPIT Technologies Ltd,613.90,607.75,602.00,622.90,600.0,700.0,2022-02-09 10:30:39,-6.15,Down
278,Jubilant Ingrevia Ltd,587.35,581.60,579.50,595.90,530.0,700.0,2022-02-09 10:30:40,-5.75,Down
279,Infosys Ltd,1717.30,1733.50,1725.00,1742.80,1700.0,2000.0,2022-02-09 10:30:41,16.20,Up
280,Tata Power Company Limited,234.85,234.35,232.60,237.65,230.0,240.0,2022-02-09 10:30:42,-0.50,Down
281,Tata Motors Limited Fully Paid Ord. Shrs,492.75,495.65,495.60,500.80,490.0,510.0,2022-02-09 10:30:43,2.90,Up
282,Coforge Ltd,4581.40,4684.35,4610.05,4724.00,4500.0,4700.0,2022-02-09 10:30:44,102.95,Up
